In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import sqlalchemy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})

In [ ]:
PATH_TO_RESULT_FILE = Path('./build/tottenham-hale-output.db').absolute()

In [ ]:
disk_engine = sqlalchemy.create_engine('sqlite:///{}'.format(PATH_TO_RESULT_FILE))

In [ ]:
def timedelta_from_iso_string(timedelta_as_string):
    t = datetime.strptime(timedelta_as_string,"PT%MM%SS")
    return timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)

assert timedelta_from_iso_string('PT19M43S') == timedelta(minutes=19) + timedelta(seconds=43)

## Read Metadata

In [ ]:
metadata = pd.read_sql_query('SELECT * FROM metadata', disk_engine, index_col='key', parse_dates=True)

In [ ]:
metadata

In [ ]:
metadata = dict(zip(metadata.index, [value[0] for value in metadata.values]))

In [ ]:
simulation_duration = timedelta_from_iso_string(metadata['durationOfSimulation'])

## Dwelling Temperature

In [ ]:
dwelling_temperature = pd.read_sql_query('SELECT * FROM temperature', disk_engine, index_col='timestamp', parse_dates=True)
dwelling_temperature.index = [pd.Timestamp.fromtimestamp(t / 1000) for t in dwelling_temperature.index]
dwelling_temperature.index.name = 'datetime'
dwelling_temperature = dwelling_temperature.pivot(columns='id')
dwelling_temperature.head()

In [ ]:
_ = sns.violinplot(dwelling_temperature.mean(), orient='vertical')
_ = plt.title('Average temperature of dwellings')
_ = plt.ylabel('Avergage temperature [˚C]')

## People Activity

In [ ]:
activity = pd.read_sql_query('SELECT * FROM activity', disk_engine, index_col='timestamp', parse_dates=True)
activity.index = [pd.Timestamp.fromtimestamp(t / 1000) for t in activity.index]
activity.index.name = 'datetime'
activity.head()

In [ ]:
sns.heatmap(pd.crosstab(activity.index, activity.value))
_ = plt.yticks([0, 144*0.25, 144*0.5, 144*0.75, 144], ["00:00", "06:00", "12:00", "18:00", "00:00"])
_ = plt.ylabel('time of day')
_ = plt.xlabel('activity')
_ = plt.title("Heatmap of people activity during the course of a day")